In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

from nc_processing import *
from analysis import * 

%matplotlib inline

In [ ]:
# define the data directory to search for files using lists
ceda_dir='/badc/cmip6/data/CMIP6/{project}/{centre}/{model}/{exp}/{run}/{domain}/{var}/gn/latest/'
# define the base of the filename for the output file(s) using lists, note this matches the file format in the data directory except for the date_4_file section and the missing .nc. 
out_base='{var}_{domain}_{model}_{exp}_{run}_gn_{time_range}' # all that's missing is a .nc but we can add that later.

In [ ]:
model='UKESM1-0-LL'
centre='MOHC'

var='tas' # near surface air temperature
domain='Amon'

exp='G6sulfur'
project='GeoMIP'

run='r1i1p1f2'

# Let's fill in the blanks in our directory structure with those variables
test_dir = ceda_dir.format(model=model, centre=centre, var=var, domain=domain, exp=exp, project=project, run=run)
# os.listdir just performs a normal "ls" command on the linux system with the text specified within the parentheses.
test_dir_files = os.listdir(test_dir)
print(test_dir_files)

In [ ]:
# specify the file path to the second file in that directory (python counts from zero for array positions.)
fpath = test_dir + test_dir_files[1]
ds = xr.open_dataset(fpath)

In [5]:
#ds
#Uncomment if you need to see what's in ds (saves screen space)

In [6]:
ds1 = xr.open_dataset(test_dir + test_dir_files[0])
ds2 = xr.open_dataset(test_dir + test_dir_files[1])

# add these datasets together concatenating along the time dimension.
ds_combined = xr.concat([ds1, ds2], 'time')

In [7]:
# choosing a time range
ds_70_100 = ds_combined.sel(time=slice('2070-01-01','2100-01-01'))
ds_70_100.time

<xarray.DataArray 'time' (time: 360)>
array([cftime.Datetime360Day(2070, 1, 16, 0, 0, 0, 0, 2, 16),
       cftime.Datetime360Day(2070, 2, 16, 0, 0, 0, 0, 4, 46),
       cftime.Datetime360Day(2070, 3, 16, 0, 0, 0, 0, 6, 76), ...,
       cftime.Datetime360Day(2099, 10, 16, 0, 0, 0, 0, 2, 286),
       cftime.Datetime360Day(2099, 11, 16, 0, 0, 0, 0, 4, 316),
       cftime.Datetime360Day(2099, 12, 16, 0, 0, 0, 0, 6, 346)], dtype=object)
Coordinates:
    height   float64 1.5
  * time     (time) object 2070-01-16 00:00:00 ... 2099-12-16 00:00:00
Attributes:
    bounds:         time_bnds
    axis:           T
    long_name:      time
    standard_name:  time

# Calculating global time mean

In [8]:
# Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 
# NOTE - to do this for another model, you'll need to look up the run number used in the piControl experiment which stores the fx variables.
# search on CEDA archive for "areacella" and "<MODEL NAME>" and it should show you.

In [9]:
# Let's see what's in the area dataset 
ds_area['areacella']
# Let's see what's in the land fraction dataset 
ds_land['sftlf']

# Let's define a data array for the area weight of the gridcells
da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
da_weight = da_weight.rename(new_name_or_name_dict='area_weight')
da_weight

<xarray.DataArray 'area_weight' (lat: 144, lon: 192)>
array([[6.197184e-07, 6.197184e-07, 6.197184e-07, ..., 6.197184e-07,
        6.197184e-07, 6.197184e-07],
       [1.858865e-06, 1.858865e-06, 1.858865e-06, ..., 1.858865e-06,
        1.858865e-06, 1.858865e-06],
       [3.097128e-06, 3.097128e-06, 3.097128e-06, ..., 3.097128e-06,
        3.097128e-06, 3.097128e-06],
       ...,
       [3.097128e-06, 3.097128e-06, 3.097128e-06, ..., 3.097128e-06,
        3.097128e-06, 3.097128e-06],
       [1.858865e-06, 1.858865e-06, 1.858865e-06, ..., 1.858865e-06,
        1.858865e-06, 1.858865e-06],
       [6.197184e-07, 6.197184e-07, 6.197184e-07, ..., 6.197184e-07,
        6.197184e-07, 6.197184e-07]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon      (lon) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1

### Finding global-mean temperature

In [10]:
data_dir='/home/users/quigley/Projects/geoengineering_summer' # !!!!!!!!!!!!! CHANGE THIS !!!!!!!!!!!!!!!!!!

model='UKESM1-0-LL'
centre='MOHC'

var='tas'
domain='Amon'

exp='G6sulfur'
project='GeoMIP'

run='r1i1p1f2'

grid='gn'

season='ANN'
dates=['2070-01-01','2100-01-01']

time_files=1

# load the arguments into a list then... 
args=[season,dates,data_dir,model,centre,var,domain,exp,project,run,grid,time_files]
ds_mean, ds_std = get_seasonal_mean_std(*args) # ... unpack them into the function.

loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN


In [11]:
# multiply each temperature value with its corresponding fraction of total area, then sum to find the global mean.
(ds_mean[var]*da_weight).sum() - 273.15 # convert from K to C.

<xarray.DataArray ()>
array(17.318323)
Coordinates:
    height   float64 ...

In [12]:
def global_mean(var,exp,dates,project):
    data_dir='/home/users/quigley/Projects/geoengineering_summer' #Changed from Pete's directory
    model='UKESM1-0-LL'
    centre='MOHC'
    domain='Amon'
    run='r1i1p1f2'
    grid='gn'
    season='ANN'
    time_files=1

    # load the arguments into a list then... 
    args=[season,dates,data_dir,model,centre,var,domain,exp,project,run,grid,time_files]
    ds_mean, ds_std = get_seasonal_mean_std(*args) # ... unpack them into the function.
    
    # multiply each temperature value with its corresponding fraction of total area, then sum to find the global mean.
    return (ds_mean[var]*da_weight).sum() - 273.15 # convert from K to C.

In [13]:
global_mean(dates=['2070-01-01','2100-01-01'],exp='G6sulfur',var='tas',project='GeoMIP')

loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN


<xarray.DataArray ()>
array(17.318323)
Coordinates:
    height   float64 ...

In [14]:
global_mean("tas","G6sulfur",['2070-01-01','2100-01-01'],"GeoMIP")

loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN


<xarray.DataArray ()>
array(17.318323)
Coordinates:
    height   float64 ...

calculate_by_metric('tas', "global_mean_anomaly_control")

In [25]:
def calculate(var, metric):
    #control_mean=global_mean(var,'historical',['1960-01-01','1990-01-01'],'CMIP') #is this modern or pre-industrial emissions?
    
    control_mean=0 #Just to test the function
    
    geo_mean=global_mean(var,'G6sulfur',['2070-01-01','2100-01-01'],'GeoMIP')  
    solar_mean=global_mean(var,'G6solar',['2070-01-01','2100-01-01'],'GeoMIP')
    sulfur_mean=global_mean(var,'G6sulfur',['2070-01-01','2100-01-01'],'GeoMIP')
    warming_mean=global_mean(var,"ssp585",['2070-01-01','2100-01-01'],"ScenarioMIP")
    
    geo_anom = geo_mean - control_mean
    sulfur_solar_anom = sulfur_mean - solar_mean
    geo_warming_anom = geo_mean - warming_mean
    warming_anom = warming_mean - control_mean
    
    if metric == "global_mean_anomaly_control":
        return geo_mean - control_mean
    elif metric == "global_mean_anomaly_solar":
        return sulfur_mean - solar_mean
    elif metric == "global_mean_anomaly_warming":
        return geo_mean - warming_mean
    elif metric == "moderated_exacerbated":
        if geo_anom < warming_anom:
            return "moderated"
        elif warming_anom < geo_anom:
            return "exacerbated"
        else:
            return "N/A"
    else:
        return 0

In [16]:
gm_anom = calculate("tas","global_mean_anomaly_control")

loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN


In [17]:
type(gm_anom)

xarray.core.dataarray.DataArray

In [18]:
gm_anom = gm_anom.values

In [19]:
gm_anom

array(17.31832275)

In [30]:
var_list = ['tas']
metric_list = ['global_mean_anomaly_control']
empty_list = []
output_array = np.array(empty_list)

for var in var_list:
    for metric in metric_list:
        #print(var,metric)
        output_array[var,metric] = calculate(var,metric).values
        
#calculate("tas","global_mean_anomaly_control")

loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_r1i1p1f2_gn_2070-01-01_2100-01-01 ANN


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [28]:
output

array(17.31832275)

In [ ]:
var_list = ['tas',"pr","huss","tasmax"]
metric_list = ['global_mean_anomaly_control',"global_mean_anomaly_solar","global_mean_anomaly_warming","moderated_exacerbated"]

dict_outer = {}
for var in var_list:
    dict_inner = {}
    for metric in metric_list:
        dict_inner[metric] = calculate(var,metric)    #CHANGE THIS!!!
    dict_outer[var] = dict_inner
    #end for metric
#end for var

df = pd.DataFrame.from_dict(dict_outer)
df.to_csv('metric_table.csv')

In [ ]:
raw_outputs = []
for var in var_list:
    for metric in metric_list:
        #raw_outputs[var, metric] = calculate(var,metric)
        type(calculate(var,metric))


In [27]:
#value_for_table = dict_inner.values[0]

TypeError: 'builtin_function_or_method' object is not subscriptable

In [38]:
ds_mean

In [40]:
for i in ds_mean:
    print(i)

time_bnds
lat_bnds
lon_bnds
tas


In [ ]:
np_area = ds_area["areacella"].values
np_land = ds_land["sftlf"].values
np_land_area = np_land * np_area